In [2]:
import math
from typing import Dict, Tuple, Optional

EPSILON = 1e-9


class Fees:
    # Placeholder for Fees, update according to your requirements
    pass


class CpmmState:
    def __init__(self, pool: Dict[str, float], p: float, collected_fees: Fees):
        self.pool = pool
        self.p = p
        self.collected_fees = collected_fees


def get_cpmm_probability(pool: Dict[str, float], p: float) -> float:
    YES, NO = pool['YES'], pool['NO']
    return (p * NO) / ((1 - p) * YES + p * NO)


def calculate_cpmm_shares(pool: Dict[str, float], p: float, bet_amount: float, bet_choice: str) -> float:
    if bet_amount == 0:
        return 0

    YES, NO = pool['YES'], pool['NO']
    k = YES ** p * NO ** (1 - p)

    if bet_choice == 'YES':
        return YES + bet_amount - (k * (bet_amount + NO) ** (p - 1)) ** (1 / p)
    else:
        return NO + bet_amount - (k * (bet_amount + YES) ** -p) ** (1 / (1 - p))


def get_cpmm_probability_after_bet_before_fees(state: CpmmState, outcome: str, bet: float) -> float:
    pool, p = state.pool, state.p
    shares = calculate_cpmm_shares(pool, p, bet, outcome)
    YES, NO = pool['YES'], pool['NO']

    if outcome == 'YES':
        new_Y, new_N = YES - shares + bet, NO + bet
    else:
        new_Y, new_N = YES + bet, NO - shares + bet

    return get_cpmm_probability({'YES': new_Y, 'NO': new_N}, p)


def get_cpmm_outcome_probability_after_bet(state: CpmmState, outcome: str, bet: float) -> float:
    new_pool = calculate_cpmm_purchase(state, bet, outcome)['new_pool']
    prob = get_cpmm_probability(new_pool, state.p)
    return 1 - prob if outcome == 'NO' else prob


def get_cpmm_fees(state: CpmmState, bet_amount: float, outcome: str) -> Tuple[float, float, Fees]:
    fee = 0
    for _ in range(10):
        bet_amount_after_fee = bet_amount - fee
        shares = calculate_cpmm_shares(state.pool, state.p, bet_amount_after_fee, outcome)
        average_prob = bet_amount_after_fee / shares
        fee = get_taker_fee(shares, average_prob)

    total_fees = 0 if bet_amount == 0 else fee
    fees = get_fees_split(total_fees, state.collected_fees)

    remaining_bet = bet_amount - total_fees

    return remaining_bet, total_fees, fees


def calculate_cpmm_purchase(state: CpmmState, bet: float, outcome: str, free_fees: Optional[bool] = False) -> dict:
    remaining_bet, fees = (bet, Fees()) if free_fees else get_cpmm_fees(state, bet, outcome)

    shares = calculate_cpmm_shares(state.pool, state.p, remaining_bet, outcome)
    YES, NO = state.pool['YES'], state.pool['NO']
    liquidity_fee = fees.liquidity_fee if hasattr(fees, 'liquidity_fee') else 0

    if outcome == 'YES':
        new_Y, new_N = YES - shares + remaining_bet + liquidity_fee, NO + remaining_bet + liquidity_fee
    else:
        new_Y, new_N = YES + remaining_bet + liquidity_fee, NO - shares + remaining_bet + liquidity_fee

    new_pool = {'YES': new_Y, 'NO': new_N}
    liquidity_data = add_cpmm_liquidity(new_pool, state.p, liquidity_fee)

    return {
        'shares': shares,
        'new_pool': liquidity_data['new_pool'],
        'new_p': liquidity_data['new_p'],
        'fees': fees
    }


def add_cpmm_liquidity(pool: Dict[str, float], p: float, amount: float) -> dict:
    prob = get_cpmm_probability(pool, p)

    YES, NO = pool['YES'], pool['NO']
    numerator = prob * (amount + YES)
    denominator = amount - NO * (prob - 1) + prob * YES
    new_p = numerator / denominator

    new_pool = {'YES': YES + amount, 'NO': NO + amount}

    old_liquidity = get_cpmm_liquidity(pool, new_p)
    new_liquidity = get_cpmm_liquidity(new_pool, new_p)
    liquidity = new_liquidity - old_liquidity

    return {'new_pool': new_pool, 'liquidity': liquidity, 'new_p': new_p}


def get_cpmm_liquidity(pool: Dict[str, float], p: float) -> float:
    YES, NO = pool['YES'], pool['NO']
    return YES ** p * NO ** (1 - p)


# Helper methods to calculate fees (these are placeholders)
def get_taker_fee(shares: float, average_prob: float) -> float:
    return shares * 0.01  # Example fee calculation


def get_fees_split(total_fees: float, collected_fees: Fees) -> Fees:
    # Example of how fees might be split, update this based on actual logic
    return Fees()


# Usage example:
state = CpmmState({'YES': 910, 'NO': 1099}, 0.55, Fees())
bet_amount = 50
outcome = 'YES'

new_prob = get_cpmm_probability_after_bet_before_fees(state, outcome, bet_amount)
print(f"New probability after bet before fees: {new_prob}")


New probability after bet before fees: 0.6154489296905195


In [4]:
class Fees:
    def __init__(self, liquidityFee=0, takerFee=0, makerFee=0):
        self.liquidityFee = liquidityFee
        self.takerFee = takerFee
        self.makerFee = makerFee

    def get_total_fees(self):
        return self.liquidityFee + self.takerFee + self.makerFee


# Helper function to split fees (you may define your own logic here)
def get_fees_split(totalFees, collectedFees):
    liquidityFee = totalFees * 0.5  # Just an example; tweak as needed
    takerFee = totalFees * 0.5
    return Fees(liquidityFee=liquidityFee, takerFee=takerFee)


# Placeholder function for taker fees
def get_taker_fee(shares, average_prob):
    return shares * average_prob * 0.01  # Example: 1% fee on the amount of shares

class CpmmState:
    def __init__(self, pool, p, collected_fees):
        self.pool = pool  # A dictionary with 'YES' and 'NO' outcomes
        self.p = p  # Probability
        self.collected_fees = collected_fees  # A Fees instance


def get_cpmm_probability(pool, p):
    yes_shares = pool['YES']
    no_shares = pool['NO']
    return (p * no_shares) / ((1 - p) * yes_shares + p * no_shares)


# Example calculation for get_cpmm_probability_after_bet_before_fees
def get_cpmm_probability_after_bet_before_fees(state, outcome, bet):
    pool = state.pool
    p = state.p
    shares = calculate_cpmm_shares(pool, p, bet, outcome)

    yes_shares = pool['YES']
    no_shares = pool['NO']

    if outcome == 'YES':
        new_yes = yes_shares - shares + bet
        new_no = no_shares + bet
    else:
        new_yes = yes_shares + bet
        new_no = no_shares - shares + bet

    return get_cpmm_probability({'YES': new_yes, 'NO': new_no}, p)


def calculate_cpmm_shares(pool, p, bet_amount, bet_choice):
    if bet_amount == 0:
        return 0

    y = pool['YES']
    n = pool['NO']
    k = y ** p * n ** (1 - p)

    if bet_choice == 'YES':
        return y + bet_amount - (k * (bet_amount + n) ** (p - 1)) ** (1 / p)
    else:
        return n + bet_amount - (k * (bet_amount + y) ** -p) ** (1 / (1 - p))


# Example usage of the fees and bet system
def calculate_cpmm_bet_example():
    # Initial state of the market
    pool = {'YES': 910, 'NO': 1099}
    p = 0.55
    collected_fees = Fees(liquidityFee=0, takerFee=0, makerFee=6.6)

    state = CpmmState(pool, p, collected_fees)

    # A user places a bet
    bet_amount = 50
    outcome = 'YES'

    # Calculate new probability before fees
    new_probability = get_cpmm_probability_after_bet_before_fees(state, outcome, bet_amount)
    print(f"New Probability after bet (before fees): {new_probability}")

    # Example fee calculation
    fees = get_fees_split(1.5, collected_fees)  # Simulate a total fee of 10 units
    print(f"Liquidity Fee: {fees.liquidityFee}, Taker Fee: {fees.takerFee}")


# Run the example calculation
calculate_cpmm_bet_example()


New Probability after bet (before fees): 0.6154489296905195
Liquidity Fee: 0.75, Taker Fee: 0.75
